In [ ]:
import pandas as pd
import plotly.express as px

### Pre každý deň by sme chceli znázorniť grafy po hodinách. Urobíme pre nástupy

In [ ]:
# nacitame vycistenu frejmu, len stlpce, co pouzijeme v tomto NB
cols = ['pick_dt', 'passengers', 'fare']
df = pd.read_parquet('data/nyc_taxi310k.parq', columns=cols)

In [ ]:
df['pick_day'] = df['pick_dt'].dt.day
df['pick_hour'] = df['pick_dt'].dt.hour
df = df.drop(columns=['pick_dt'])

In [ ]:
# Potrebujeme zoskupit data podla dni a hodin; groupped sama o sebe je nam netreba
groupped = df.groupby(['pick_day', 'pick_hour'])
type(groupped)

In [ ]:
df_days = groupped['passengers'].aggregate("sum").reset_index()
df_days = df_days.rename(columns={'passengers': 'pass_count'})
df_days.head()

In [ ]:
df_days['fares_count'] = groupped['fare'].count().reset_index()['fare']
df_days['total_fare'] = groupped['fare'].sum().reset_index()['fare']
df_days.head()

In [ ]:
# ako vyberieme jeden den? lahko, napr. 14. jan.
df_days.filter(pl.col('pick_day') == 14).sort(by='pick_hour').head()
df_days.columns

In [ ]:
# funkcia pre vyrobenie df_days z povodnej df; urobime grupovanie a agregacie na jeden sup
def daily_frame(frm):   # vyvolame s df ako frm
    groupped = df.groupby(['pick_day', 'pick_hour'])
    df_days = groupped['passengers'].aggregate("sum").reset_index()
    df_days = df_days.rename(columns={'passengers': 'pass_count'})
    df_days['fares_count'] = groupped['fare'].count().reset_index()['fare']
    df_days['total_fare'] = groupped['fare'].sum().reset_index()['fare']
    return df_days

In [ ]:
# mame vsetko, aby sme kreslili, podobne bude ako monthly_plot z data_funkcie
def daily_plot(day): # frm tu bude povyssia df_days
    frm_d = daily_frame(df)
    frm_day = frm_d[frm_d['pick_day'] == day].sort_values(by='pick_hour')
    pass_fares_plot = px.bar(frm_day, x='pick_hour', y=['pass_count', 'fares_count'], barmode='group',
                             labels={'pick_hour': 'Hodina', 'value': 'Hodnoty', 'variable': 'Premenná'})
    pass_fares_plot.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(24))))
    return pass_fares_plot

In [ ]:
daily_plot(26)

#### Obe funkcie, daily_frame aj daily_plot nakopčíme do `data_funkcie.py` pre budúce užitie